## Importar librerías

In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

### Carga de documento y split

In [2]:
# Cargar el documento
loader = TextLoader('Fuentes datos/Historia España.txt', encoding="utf8")
documents = loader.load()

In [3]:
# Dividir en chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500) #Otro método de split basándose en tokens
docs = text_splitter.split_documents(documents)

Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 1009, which is longer than the specified 500
Created a chunk of size 2228, which is longer than the specified 500


### Conectar a OpenAI para los embeddings

In [4]:
f = open('/home/iabd/clave_gpt.txt')
api_key = f.read().strip()

In [5]:
funcion_embedding = OpenAIEmbeddings(openai_api_key=api_key)

# Alternativa con SKLearn Vector Store

In [6]:
from langchain_community.vectorstores import SKLearnVectorStore #pip install scikit-learn / pip install pandas pyarrow

In [7]:
persist_path="./ejemplosk_embedding_db"  #ruta donde se guardará la BBDD vectorizada

#Creamos la BBDD de vectores a partir de los documentos y la función embeddings
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=funcion_embedding,
    persist_path=persist_path,
    serializer="parquet", #el serializador o formato de la BD lo definimos como parquet
)

In [8]:
# Fuerza a guardar los nuevos embeddings en el disco
vector_store.persist()

In [9]:
#Creamos un nuevo documento que será nuestra "consulta" para buscar el de mayor similitud en nuestra Base de Datos de Vectores y devolverlo
consulta = "dame información de la Primera Guerra Mundial"
docs = vector_store.similarity_search(consulta)
print(docs[0].page_content)

Primera Guerra Mundial
Artículos principales: España en la Primera Guerra Mundial y Economía de España durante la Primera Guerra Mundial.
La neutralidad de España en la Primera Guerra Mundial le permitió convertirse en un proveedor de material para los contendientes, lo que provocó una burbuja económica durante los años de la contienda. La Guerra del Rif, el brote de la gripe de 1918 en España y en otros lugares, junto con una desaceleración económica importante en el período de posguerra, llegó a España particularmente difícil, y el país entró en crisis. Como intento por superar dicha situación, el rey Alfonso XIII decidió apoyar a la dictadura del general Miguel Primo de Rivera.

Crisis de la Restauración
Artículos principales: Crisis de la Restauración, Crisis española de 1917 y Trienio Bolchevique.
A partir de 1917, se constata una descomposición interna del régimen político canovista de la restauración, caracterizado por la atomización en bandos personalista de los partidos políti

## Cargar la BD de vectores (uso posterior una vez tenemos creada ya la BD)

In [10]:
vector_store_connection = SKLearnVectorStore(
    embedding=funcion_embedding, persist_path=persist_path, serializer="parquet"
)
print("Una instancia de la BBDD de vectores se ha cargado desde ", persist_path)

Una instancia de la BBDD de vectores se ha cargado desde  ./ejemplosk_embedding_db


In [11]:
vector_store_connection

In [12]:
nueva_consulta = "¿Qué paso en el siglo de Oro?"

In [13]:
docs = vector_store_connection.similarity_search(nueva_consulta)
print(docs[0].page_content)

El área de influencia de España se expandió, constituyéndose en la mayor potencia económica del mundo durante el siglo xvi, el comercio floreció a través del Atlántico entre la península ibérica y las Américas, y en el Pacífico desde Asia del Este y las Filipinas hasta México, y en el aspecto militar, durante varios siglos el Imperio español dominaría los mares y océanos con la armada y los campos de batallas con la infantería de los tercios. Aunque a partir del siglo xvii su poder e influencia en el centro de Europa e Italia se vio ampliamente contestado.

Durante su apogeo económico, el imperio tuvo gran prestigio cultural e influencia militar. Muchas cosas que provenían de España eran a menudo imitadas. Las expresiones artísticas más cultivadas en España fueron la literatura, las artes plásticas, la música y la arquitectura, mientras el saber se acumulaba y se enseñaba desde prestigiosas universidades como las de Salamanca o Alcalá de Henares. Sin embargo, a diferencia de lo que pas